## ETL Project -  Top hits

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import kaggle
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Open browser with splinter (will perform two separate site searches in the same browser)
# (Headless = True) Thus we do not see the browser oepning and closing
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

## Insider - Scraping

In [3]:
# HTML Object and Beautiful Soup Parseing
url = 'https://www.insider.com/best-songs-every-year-2017-8'
browser.visit(url)

In [4]:
# HTML Object and Beautiful Soup Parseing
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# Retrieve and collect all desired data
insider = soup.find_all('h2', class_='slide-title-text')

# All desired data is in one line, thus have to split
insider_list = []
temp_list = []

for i in insider:
    title= i.text.split(":")[1]
    title = title.split("—")[0]
    artist = i.text.split("—")[1]
    year = i.text.split(":")[0]
    temp_list.append(title)
    temp_list.append(artist)
    temp_list.append(year)
    insider_list.append(temp_list)
    temp_list = []
    
print(insider_list)

[[' "I\'ll Never Smile Again" ', ' Tommy Dorsey', '1940'], [' "Amapola (Pretty Little Poppy)" ', ' Jimmy Dorsey', '1941'], [' "White Christmas" ', ' Bing Crosby', '1942'], [' "I\'ve Heard That Song Before" ', ' Harry James', '1943'], [' "Swinging on a Star" ', ' Bing Crosby', '1944'], [' "\'Till the End of Time" ', ' Perry Como', '1945'], [' "The Gypsy" ', ' The Ink Spots', '1946'], [' "Near You" ', ' Francis Craig', '1947'], [' "Buttons and Bows" ', ' Dinah Shore', '1948'], [' "Riders in the Sky (A Cowboy Legend) ', ' Vaughn Monroe', '1949'], [' "Goodnight, Irene" ', ' Gordon Jenkins and The Weavers', '1950'], [' "Too Young" ', ' Nat King Cole', '1951'], [' "Blue Tango" ', ' Leroy Anderson', '1952'], [' "The Song From Moulin Rouge (Where Is Your Heart)" ', ' Percy Faith', '1953'], [' "Little Things Mean A Lot" ', ' Kitty Kallen', '1954'], [' "Cherry Pink (and Apple Blossom White)"', ' Pérez Prado', '1955'], [' "Heartbreak Hotel" ', ' Elvis Presley', '1956'], [' "All Shook Up" ', ' Elv

In [6]:
# Convert data into df and clean
insider_df = pd.DataFrame(insider_list, columns = ["Title","Artist", "Year"])
insider_df["Source"] = "Insider #1 Song of Year"

# Convert years to integers 
insider_df['Year'] = (insider_df['Year']).astype(int)

# Removing extra years 
insider_df = insider_df[insider_df['Year'] <= 2019]
insider_df = insider_df[insider_df['Year'] >= 2010]
insider_df = insider_df.sort_values(by=['Year'])

insider_df

,Title,Artist,Year,Source
70,"""TiK ToK""",Kesha,2010,Insider #1 Song of Year
71,"""Rolling In The Deep""",Adele,2011,Insider #1 Song of Year
72,"""Somebody That I Used To Know""",Gotye featuring Kimbra,2012,Insider #1 Song of Year
73,"""Thrift Shop""",Macklemore & Ryan Lewis featuring Wanz,2013,Insider #1 Song of Year
74,"""Happy""",Pharrell Williams,2014,Insider #1 Song of Year
75,"""See You Again""",Wiz Khalifa featuring Charlie Puth,2015,Insider #1 Song of Year
76,"""Love Yourself""",Justin Bieber,2016,Insider #1 Song of Year
77,"""Despacito""",Luis Fonsi and Daddy Yankee featuring Justin ...,2017,Insider #1 Song of Year
78,"""God's Plan""",Drake,2018,Insider #1 Song of Year


## Billboard - Scraping

In [7]:
# HTML Object and Beautiful Soup Parseing
url = 'https://www.billboard.com/charts/decade-end/hot-100'
browser.visit(url)

In [8]:
# HTML Object and Beautiful Soup Parseing
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

There are adds and various class names, thus not able to simply sort and pull.
Pulled out each desired object separetly and then merged three dataframes into one

In [9]:
# Scrape title data
title = soup.find_all('div', class_='ye-chart-item__title')

# Put each title into a list of lists
title_list = []
count = 0

for i in title:
    new_list = []
    new_list.append(count)
    new_list.append(i.text.replace("\n",""))
    title_list.append(new_list)
    count += 1
    
# Convert title lsit to df
title_df = pd.DataFrame(title_list, columns = ["key","Title"])
title_df.head()

,key,Title
0,0,Uptown Funk!
1,1,Party Rock Anthem
2,2,Shape Of You
3,3,Closer
4,4,Girls Like You


In [10]:
# Scrape artist data
artist = soup.find_all('div', class_='ye-chart-item__artist')

# Put each artist into a list of lists
artist_list = []
count = 0

for i in artist:
    new_list = []
    new_list.append(count)
    new_list.append(i.text.replace("\n",""))
    artist_list.append(new_list)
    count += 1

# Convert artist list to df
artist_df = pd.DataFrame(artist_list, columns = ["key","Artist"])
artist_df.head()

,key,Artist
0,0,Mark Ronson Featuring Bruno Mars
1,1,LMFAO Featuring Lauren Bennett & GoonRock
2,2,Ed Sheeran
3,3,The Chainsmokers Featuring Halsey
4,4,Maroon 5 Featuring Cardi B


In [11]:
# Scrape date data
date = soup.find_all('span', class_='decade-end-chart-item__peak-info-date')

# Put each date into a list of lists
date_list = []
count = 0

for i in date:
    new_list = []
    new_list.append(count)
    new_list.append(i.text.replace("\n",""))
    date_list.append(new_list)
    count += 1
    
# Convert date list to df
date_df = pd.DataFrame(date_list, columns = ["key","Date"])
date_df = date_df['Date'].str[:4].reset_index()
date_df.columns=["key", "Year"]
date_df.head()

,key,Year
0,0,2015
1,1,2011
2,2,2017
3,3,2016
4,4,2018


In [12]:
# Merge all dfs into one for Billboard results
billboard_partial_df = pd.merge(title_df, artist_df, on="key")
billboard_df = pd.merge(billboard_partial_df, date_df, on="key")

# Clean and add souce column 
del billboard_df["key"]
billboard_df["Source"] = "Billboard Top 100"

# Convert years to integers 
billboard_df['Year'] = (billboard_df['Year']).astype(int)

# Removing extra years 
billboard_df = billboard_df[billboard_df['Year'] <= 2018]
billboard_df = billboard_df[billboard_df['Year'] >= 2010]
billboard_df = billboard_df.sort_values(by=['Year'])

billboard_df.head()

,Title,Artist,Year,Source
49,Dynamite,Taio Cruz,2010,Billboard Top 100
42,Firework,Katy Perry,2010,Billboard Top 100
23,TiK ToK,Ke$ha,2010,Billboard Top 100
21,Just The Way You Are,Bruno Mars,2010,Billboard Top 100
45,Love The Way You Lie,Eminem Featuring Rihanna,2010,Billboard Top 100


In [13]:
# Finished scraping from the web, quit browser in background
browser.quit()

## Spotify/ Kaggle - csv

In [14]:
#API command used from Kaggle:
kaggle_api = 'leonardopena/top-spotify-songs-from-20102019-by-year'

# Import kaggle and place cvs into resources folder
kaggle.api.authenticate()
kaggle.api.dataset_download_files(kaggle_api, unzip=True, path='./Resources', quiet=True)

In [15]:
#Read file and store into pandas data frame
spotify_raw_csv = "./Resources/top10s.csv"
spotify_2010_2019_df = pd.read_csv(spotify_raw_csv, sep = ',', encoding='ISO-8859-1')

# Display head of data set to be sure it was loaded correctly
spotify_2010_2019_df.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [16]:
# Clean df
spotify_clean_df = spotify_2010_2019_df[['title','artist' ,'year']].copy()
spotify_clean_df.columns = ['Title', 'Artist', 'Year']
spotify_clean_df['Source']='Spotify'

# Removing extra years 
spotify_clean_df = spotify_clean_df[spotify_clean_df['Year'] < 2019]
spotify_clean_df.head()

,Title,Artist,Year,Source
0,"Hey, Soul Sister",Train,2010,Spotify
1,Love The Way You Lie,Eminem,2010,Spotify
2,TiK ToK,Kesha,2010,Spotify
3,Bad Romance,Lady Gaga,2010,Spotify
4,Just the Way You Are,Bruno Mars,2010,Spotify


### Connect to local database

In [17]:
# Connect to local database
code_source_passcode = open('/Users/Richa/Desktop/SQL_private_connect.py') 
sql_private_connect = code_source_passcode.read()

In [18]:
engine = create_engine(f'postgresql://{sql_private_connect}@localhost:5432/music_tunes_db')
connection = engine.connect()

In [19]:
# Check for tables
engine.table_names() 

['combined_music', 'insider', 'billboard_top', 'spotify']

In [20]:
# Use pandas to load json converted DataFrame into database
insider_df.to_sql(name='insider', 
                            con=engine, 
                            if_exists='replace', 
                            index=False)

In [21]:
# Use pandas to load json converted DataFrame into database
billboard_df.to_sql(name='billboard_top', 
                                con=engine, 
                                if_exists='replace', 
                                index=False)

In [22]:
# Use pandas to load csv converted DataFrame into database
spotify_clean_df.to_sql(name='spotify', 
                                con=engine, 
                                if_exists='replace', 
                                index=False)

In [23]:
# Check for tables
engine.table_names() 

['combined_music', 'insider', 'billboard_top', 'spotify']

In [24]:
# Confirm data has been added by querying the insider table
insider_loaded = pd.read_sql_query('select * from insider', con=engine)
insider_loaded.head()

,Title,Artist,Year,Source
0,"""TiK ToK""",Kesha,2010,Insider #1 Song of Year
1,"""Rolling In The Deep""",Adele,2011,Insider #1 Song of Year
2,"""Somebody That I Used To Know""",Gotye featuring Kimbra,2012,Insider #1 Song of Year
3,"""Thrift Shop""",Macklemore & Ryan Lewis featuring Wanz,2013,Insider #1 Song of Year
4,"""Happy""",Pharrell Williams,2014,Insider #1 Song of Year


In [25]:
# Confirm data has been added by querying the billboard_top table
billboard_loaded = pd.read_sql_query('select * from billboard_top', con=engine)
billboard_loaded.head()

,Title,Artist,Year,Source
0,Dynamite,Taio Cruz,2010,Billboard Top 100
1,Firework,Katy Perry,2010,Billboard Top 100
2,TiK ToK,Ke$ha,2010,Billboard Top 100
3,Just The Way You Are,Bruno Mars,2010,Billboard Top 100
4,Love The Way You Lie,Eminem Featuring Rihanna,2010,Billboard Top 100


In [26]:
# Confirm data has been added by querying the spotify table
spotify_loaded = pd.read_sql_query('select * from spotify', con=engine)
spotify_loaded.head()

,Title,Artist,Year,Source
0,"Hey, Soul Sister",Train,2010,Spotify
1,Love The Way You Lie,Eminem,2010,Spotify
2,TiK ToK,Kesha,2010,Spotify
3,Bad Romance,Lady Gaga,2010,Spotify
4,Just the Way You Are,Bruno Mars,2010,Spotify


## Show Merged Data

In [27]:
# Performed a UNION in pgAdmin to merge the data into one dataframe
combined_loaded_data = pd.read_sql_query('select * from combined_music', con=engine)
combined_loaded_data.head()

,title,artist,year,source
0,People Like Us,Kelly Clarkson,2013,Spotify
1,Bad Romance,Lady Gaga,2010,Spotify
2,Titanium (feat. Sia),David Guetta,2012,Spotify
3,What Lovers Do (feat. SZA),Maroon 5,2018,Spotify
4,Want To,Dua Lipa,2018,Spotify


In [28]:
# Check all data was added to combined
print(len(insider_loaded)+len(billboard_loaded)+len(spotify_loaded))
print(len(combined_loaded_data))

672
671


We lost a title in the csv file when performing the union. Need more time to find which artist is not transferring and whiy. There was trouble encoding the csv file as well.


In [29]:
list_of_df = [insider_loaded, billboard_loaded, spotify_loaded]
pd_combine = pd.concat(list_of_df)
pd_combine.head()

,Title,Artist,Year,Source
0,"""TiK ToK""",Kesha,2010,Insider #1 Song of Year
1,"""Rolling In The Deep""",Adele,2011,Insider #1 Song of Year
2,"""Somebody That I Used To Know""",Gotye featuring Kimbra,2012,Insider #1 Song of Year
3,"""Thrift Shop""",Macklemore & Ryan Lewis featuring Wanz,2013,Insider #1 Song of Year
4,"""Happy""",Pharrell Williams,2014,Insider #1 Song of Year


When running the merge through pandas all data is transferred correctly

In [30]:
print(len(insider_loaded)+len(billboard_loaded)+len(spotify_loaded))
print(len(pd_combine))

672
672


## Analyse Data

In [31]:
# Check if any titles are shared between the three checked sources
boolean = pd_combine['Title'].duplicated().any()
boolean

True

In [32]:
# Show all song titles that were repeated 
duplicate_df = pd.concat(g for _, g in pd_combine.groupby("Title") if len(g) > 1)
duplicate_df.head(10)

,Title,Artist,Year,Source
178,A Little Party Never Killed Nobody (All We Got),Fergie,2013,Spotify
239,A Little Party Never Killed Nobody (All We Got),Fergie,2014,Spotify
41,All About That Bass,Meghan Trainor,2014,Billboard Top 100
324,All About That Bass,Meghan Trainor,2015,Spotify
398,All I Ask,Adele,2016,Spotify
473,All I Ask,Adele,2017,Spotify
31,Blurred Lines,Robin Thicke Featuring T.I. + Pharrell,2013,Billboard Top 100
150,Blurred Lines,Robin Thicke,2013,Spotify
10,Break Your Heart,Taio Cruz Featuring Ludacris,2010,Billboard Top 100
49,Break Your Heart,Taio Cruz,2010,Spotify
